In [ ]:
!pip install konlpy
!pip install wordcloud

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 41.0 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (10.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unab

In [ ]:
import re
from datetime import datetime
import collections

import requests, json
from bs4 import BeautifulSoup
import urllib.parse as parse

from tqdm import tqdm
import pickle

import numpy as np
import pandas as pd
import sklearn as sk

from konlpy.tag import Okt # komoran, hannanum, kkma, mecab
import gensim
from gensim.summarization import summarize

#visualization
import matplotlib as mpl
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rcParams
import seaborn as sns
from wordcloud import WordCloud

# 브라우저에서 바로 그려지도록 
# %matplotlib inline

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# # Colab 의 한글 폰트 설정
# plt.rc('font', family='NanumSquareRound') 

# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

# Seaborn 테마 설정
sns.set_theme(style="ticks", color_codes=True, font='NanumSquareRound', font_scale=2)

# 워닝 메세지 삭제
import warnings
warnings.filterwarnings(action='ignore')

# font_list = [font.name for font in fm.fontManager.ttflist]
# font_list

In [ ]:
font_list = [font.name for font in fm.fontManager.ttflist]
font_list
path = '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf'

In [ ]:
df0 = pd.read_csv('러시아 한국 경제_v2_2019041520180911.csv')
df1 = pd.read_csv('러시아 한국 경제_v2_2022041520220224.csv')


In [ ]:
df0.shape, df1.shape

((1456, 4), (1456, 4))

In [ ]:
df0.dropna(inplace=True)
df1.dropna(inplace=True)

데이터 전처리

In [ ]:
def clean_content(content):
  stripped = content[69:-2]
  cleaned = re.sub('[^가-힣ㄱ-ㅣa-zA-Z|0-9.%]', ' ', string=stripped)
  return cleaned

def clean_title(title):
  cleaned = re.sub('[^가-힣ㄱ-ㅣa-zA-Z|0-9.%]', ' ', string=title)
  return cleaned

In [ ]:
df0['내용'] = df0['내용'].apply(clean_content)
df1['내용'] = df1['내용'].apply(clean_content)

In [ ]:
df0['제목'] = df0['제목'].apply(clean_title) + '.'
df1['제목'] = df1['제목'].apply(clean_title) + '.'

In [ ]:
asdf0 = df0.drop(['등록일시','주소'],axis =1) # 불필요 컬럼 삭제

asdf1 = df1.drop(['등록일시','주소'],axis =1)

In [ ]:
before = asdf0   # 2019년 기사 : before
after = asdf1    # 2022년 기사 : after

라벨 부여

In [ ]:
before['label'] = 1
after['label'] = 0

In [ ]:
before['제목내용'] = before['제목'] + ' ' + before['내용']
after['제목내용'] = after['제목'] + ' ' + after['내용']

자료 통합

In [ ]:
tog = pd.concat([before] + [after]) # together = tog

In [ ]:
tog = tog.reset_index() # 리셋 인덱스 후 인덱스 항목 추가

In [ ]:
len(tog)

2912

In [ ]:
tog

,index,제목,내용,label,제목내용
0,0,정운찬 칼럼 우리가 세워야 할 새로운 나라.,한강을 굽어보는 국립현충원에 봄이 가득하다. 하얀 목련이 고결한 자태를 뽐내고 홍...,1,정운찬 칼럼 우리가 세워야 할 새로운 나라. 한강을 굽어보는 국립현충원에 봄이 가...
1,1,자본 빨아들이는 북미 셰일가스 좋은 투자 기회 생겨날 것 .,글로벌 에너지PE 퍼스트리저브 알렉스 크루거 CEO3 4년 걸리는 유전개발과 달리6...,1,자본 빨아들이는 북미 셰일가스 좋은 투자 기회 생겨날 것 . 글로벌 에너지PE 퍼...
2,2,중앙아시아 중심 카자흐스탄 기계 인프라 서비스 경제 협력 유망 .,한국과 카자흐스탄의 경제협력 유망 분야로 기계 인프라 서비스 등이 꼽혔다....,1,중앙아시아 중심 카자흐스탄 기계 인프라 서비스 경제 협력 유망 . 한국...
3,3,김대성의 환율이야기 노르웨이 국부펀드가 한국 채권 배제한 교훈.,자료 한국은행 제공노르웨이 국부펀드가 한국 채권을 포트폴리오에서 제외키로 결정한...,1,김대성의 환율이야기 노르웨이 국부펀드가 한국 채권 배제한 교훈. 자료 한국은...
4,4,베네수엘라 아르헨 그리스 나랏 돈 펑펑 쓰다 경제 파탄.,Cover Story 국가 부채 사상 최대선심성 복지지출 등 포퓰리즘에 재정 무...,1,베네수엘라 아르헨 그리스 나랏 돈 펑펑 쓰다 경제 파탄. Cover Story ...
...,...,...,...,...,...
2907,1451,전국 주요 신문 톱뉴스 24일 조간 .,경향신문 미 러에 본격 제재 대화 문도 닫혔다 국민일보 ...,0,전국 주요 신문 톱뉴스 24일 조간 . 경향신문 미 러에 본격 제...
2908,1452,조간브리핑 고신용자도 대출 금리 4% 넘어서 본격적 고금리 부담 시작.,ack 경제와이드 모닝벨 조간 브리핑 장연재조간브리핑입니다. ...,0,조간브리핑 고신용자도 대출 금리 4% 넘어서 본격적 고금리 부담 시작. ac...
2909,1453,사라지지 않은 러시아 전면 침공 가능성 미 기준금리 전망도 움직이나 글로벌마켓...,ack 앵커 오늘 미국 증시 흐름에서 가장 특징적인 부분부터 짚어...,0,사라지지 않은 러시아 전면 침공 가능성 미 기준금리 전망도 움직이나 글로벌마켓...
2910,1454,국제유가 장중 99달러 한국 3%성장 빨간불 .,우크라이나 사태 올 전망치 3.1%는 73달러 전제러시아의 우크라이나 침공 소식에...,0,국제유가 장중 99달러 한국 3%성장 빨간불 . 우크라이나 사태 올 전망치 ...


학습용과 검증용으로 분리

In [ ]:
test1 = tog[(tog['index'] <= 300)] # 통합 파일에서 index 300 아래 데이터 추출 (검증용)

In [ ]:
test = test1.drop([test1.columns[0],test1.columns[1],test1.columns[2]], axis=1)# 검증용으로 602개 분리 후 불필요 칼럼 제거
test

,label,제목내용
0,1,정운찬 칼럼 우리가 세워야 할 새로운 나라. 한강을 굽어보는 국립현충원에 봄이 가...
1,1,자본 빨아들이는 북미 셰일가스 좋은 투자 기회 생겨날 것 . 글로벌 에너지PE 퍼...
2,1,중앙아시아 중심 카자흐스탄 기계 인프라 서비스 경제 협력 유망 . 한국...
3,1,김대성의 환율이야기 노르웨이 국부펀드가 한국 채권 배제한 교훈. 자료 한국은...
4,1,베네수엘라 아르헨 그리스 나랏 돈 펑펑 쓰다 경제 파탄. Cover Story ...
...,...,...
1752,0,원투펀치 맞은 한국경제 올해 성장률 3% 달성 힘들다 . 대외 의존도 높아 역대...
1753,0,10년새 경험못했던 i 쓰나미 가 닥친다. 우크라 전쟁 상하이항 봉쇄로 원자재 ...
1754,0,승승장구하는 K방산 주가도 꿈틀 . ack 기자 영하 30도...
1755,0,다시 나는 항공편 완전한 회복까지는 먼 길 . ack 앵커 다음...


In [ ]:
test.columns

Index(['label', '제목내용'], dtype='object')

In [ ]:
train1 = tog[(tog['index'] < 300)].index # 검증용 파일로 추출한 데이터 삭제
train = tog.drop(train1)

In [ ]:
train1 = train.reset_index()

In [ ]:
train = train1.drop([train1.columns[0],train1.columns[1],train1.columns[2],train1.columns[3]], axis=1) # 학습용으로 2312개 분리 후 불필요 칼럼 제거
train

,label,제목내용
0,1,한국 전쟁수행능력 일본을 앞섰다 . 정욱식 칼럼 경제력 차이 3배 국방 예산...
1,1,기업을 UP 시켜라 3%에 묶인 대주주 의결권 떼지어 오는 헤지펀드 어떻게 ...
2,1,산업연구원 러 극동서 남 북 러 3각 경협 추진 미리 준비해야 . 한반도 평화 정...
3,1,올림픽 첫 금메달 때 포니... 박항서 기적 때 베트남 첫 완성차. 197...
4,1,50 60 댓글 달지 말고 아세안 가라 는 김현철 경제보좌관은 누구. 현 정부 대...
...,...,...
2307,0,전국 주요 신문 톱뉴스 24일 조간 . 경향신문 미 러에 본격 제...
2308,0,조간브리핑 고신용자도 대출 금리 4% 넘어서 본격적 고금리 부담 시작. ac...
2309,0,사라지지 않은 러시아 전면 침공 가능성 미 기준금리 전망도 움직이나 글로벌마켓...
2310,0,국제유가 장중 99달러 한국 3%성장 빨간불 . 우크라이나 사태 올 전망치 ...


데이터 피클로 저장


In [ ]:
tog = tog.drop([tog.columns[0],tog.columns[1],tog.columns[2]], axis=1)
tog

,label,제목내용
0,1,정운찬 칼럼 우리가 세워야 할 새로운 나라. 한강을 굽어보는 국립현충원에 봄이 가...
1,1,자본 빨아들이는 북미 셰일가스 좋은 투자 기회 생겨날 것 . 글로벌 에너지PE 퍼...
2,1,중앙아시아 중심 카자흐스탄 기계 인프라 서비스 경제 협력 유망 . 한국...
3,1,김대성의 환율이야기 노르웨이 국부펀드가 한국 채권 배제한 교훈. 자료 한국은...
4,1,베네수엘라 아르헨 그리스 나랏 돈 펑펑 쓰다 경제 파탄. Cover Story ...
...,...,...
2907,0,전국 주요 신문 톱뉴스 24일 조간 . 경향신문 미 러에 본격 제...
2908,0,조간브리핑 고신용자도 대출 금리 4% 넘어서 본격적 고금리 부담 시작. ac...
2909,0,사라지지 않은 러시아 전면 침공 가능성 미 기준금리 전망도 움직이나 글로벌마켓...
2910,0,국제유가 장중 99달러 한국 3%성장 빨간불 . 우크라이나 사태 올 전망치 ...


In [ ]:
import pickle
with open('tog.pickle','wb')as fw:
    pickle.dump(tog,fw)
with open('tog.pickle','rb')as fr:
    tog = pickle.load(fr)
print(tog)

      label                                               제목내용
0         1   정운찬 칼럼 우리가 세워야 할 새로운 나라. 한강을 굽어보는 국립현충원에 봄이 가...
1         1   자본 빨아들이는 북미 셰일가스 좋은 투자 기회 생겨날 것 . 글로벌 에너지PE 퍼...
2         1   중앙아시아 중심 카자흐스탄  기계  인프라  서비스 경제 협력 유망 .    한국...
3         1   김대성의 환율이야기  노르웨이 국부펀드가 한국 채권 배제한 교훈.   자료 한국은...
4         1  베네수엘라 아르헨 그리스 나랏 돈 펑펑 쓰다 경제 파탄. Cover Story   ...
...     ...                                                ...
2907      0   전국 주요 신문 톱뉴스  24일 조간 .      경향신문   미  러에 본격 제...
2908      0   조간브리핑  고신용자도 대출 금리 4% 넘어서 본격적  고금리 부담  시작. ac...
2909      0  사라지지 않은 러시아 전면 침공 가능성   미 기준금리 전망도 움직이나  글로벌마켓...
2910      0  국제유가 장중 99달러  한국  3%성장  빨간불 .  우크라이나 사태 올 전망치 ...
2911      0   View   Review  커지는 우크라 리스크 원유 수입 5위 한국  문제는 에...

[2912 rows x 2 columns]


In [ ]:
import pickle
with open('train.pickle','wb')as fw:
    pickle.dump(train,fw)
with open('train.pickle','rb')as fr:
    train = pickle.load(fr)
print(train)

      label                                               제목내용
0         1  한국 전쟁수행능력  일본을 앞섰다 .  정욱식 칼럼  경제력 차이 3배  국방 예산...
1         1   기업을   UP 시켜라  3%에 묶인 대주주 의결권 떼지어 오는 헤지펀드 어떻게 ...
2         1  산업연구원  러 극동서 남 북 러 3각 경협 추진 미리 준비해야 . 한반도 평화 정...
3         1   올림픽 첫 금메달  때   포니... 박항서 기적  때 베트남 첫 완성차. 197...
4         1   50 60 댓글 달지 말고 아세안 가라 는 김현철 경제보좌관은 누구. 현 정부 대...
...     ...                                                ...
2307      0   전국 주요 신문 톱뉴스  24일 조간 .      경향신문   미  러에 본격 제...
2308      0   조간브리핑  고신용자도 대출 금리 4% 넘어서 본격적  고금리 부담  시작. ac...
2309      0  사라지지 않은 러시아 전면 침공 가능성   미 기준금리 전망도 움직이나  글로벌마켓...
2310      0  국제유가 장중 99달러  한국  3%성장  빨간불 .  우크라이나 사태 올 전망치 ...
2311      0   View   Review  커지는 우크라 리스크 원유 수입 5위 한국  문제는 에...

[2312 rows x 2 columns]


In [ ]:
with open('test.pickle','wb')as fw:
    pickle.dump(test,fw)
with open('test.pickle','rb')as fr:
    test = pickle.load(fr)
print(test)

      label                                               제목내용
0         1   정운찬 칼럼 우리가 세워야 할 새로운 나라. 한강을 굽어보는 국립현충원에 봄이 가...
1         1   자본 빨아들이는 북미 셰일가스 좋은 투자 기회 생겨날 것 . 글로벌 에너지PE 퍼...
2         1   중앙아시아 중심 카자흐스탄  기계  인프라  서비스 경제 협력 유망 .    한국...
3         1   김대성의 환율이야기  노르웨이 국부펀드가 한국 채권 배제한 교훈.   자료 한국은...
4         1  베네수엘라 아르헨 그리스 나랏 돈 펑펑 쓰다 경제 파탄. Cover Story   ...
...     ...                                                ...
1752      0  원투펀치 맞은 한국경제   올해 성장률 3% 달성 힘들다 . 대외 의존도 높아 역대...
1753      0  10년새 경험못했던   i 쓰나미 가 닥친다. 우크라 전쟁 상하이항 봉쇄로 원자재 ...
1754      0  승승장구하는 K방산   주가도  꿈틀 . ack           기자 영하 30도...
1755      0  다시 나는 항공편  완전한 회복까지는 먼 길 . ack           앵커 다음...
1756      0  ADB    물가상승률 1.9% 3.2%로 대폭 상향. 우크라사태 글로벌 인플레 영...

[602 rows x 2 columns]


In [ ]:
X_train = train['제목내용']
y_train = train.label

X_test = test['제목내용']  
y_test = test.label 
X_train[0], y_train[0]

('한국 전쟁수행능력  일본을 앞섰다 .  정욱식 칼럼  경제력 차이 3배  국방 예산 과도하다는 증거  정욱식 평화네트워크 대표  군사력 측정 전문기관인 글로벌 파이어 파워 GFP 가  2018년 잠재적인 전쟁 수행능력 에서 한국이 일본을 앞섰다는 평가를 내놨다. 한국을 136개 평가 대상국 가운데 7위로 평가한 반면 일본은 8위를 기록했다는 것이다. 이 기관이 한국의 전쟁 수행능력을 일본보다 높게 평가한 것은 2006년 이래 처음이다.GFP는 군사력의 양과 질적 측면뿐만 아니라 자원  재정  지리  산업 등 55개 요소로 해당 국가의 전쟁 수행능력을 평가해왔다. 하지만 핵무기 보유고는 중요한 요소에 포함시키지 않았다. 재래식 군사력을 동원한 전쟁 수행 능력을 주된 평가 대상으로 삼았기 때문이다. 이에 따라 비공식 핵보유국들인 이스라엘  파키스탄  북한의 전쟁 수행능력은 각각 16  17  18위로 평가되었다.   글로벌 파이어 파워 가 발표한  2018년 잠재적인 전쟁 수행 능력  순위  www.globalfirepower.com일본을 앞선 핵심 사유는  가용 병력 GFP는 0.0000에 가까울수록 전쟁 능력이 우수한 것이라며  한국은 0.2001로  일본은 0.2107로 점수를 매겼다. 두 나라 앞에는 미국  러시아  중국  인도  프랑스  영국 순으로 순위가 매겨졌다.GFP가 한국을 일본보다 높은 순위에 올려놓은 핵심적인 이유는 정규군과 예비군을 합친  가용 병력 의 차이에 있다. 일본은 약 31만 명인 반면에 한국은 17배에 달하는 약 580만 명에 해당된다는 것이다. 정규군 규모에 있어서도 일본은 약 24만 명인 반면에  한국은 일본보다 2배가 훨씬 넘는 60만 명 정도이다.한편 이 기관은 한국과 일본의 군용기 보유 수는 1560대와 1508대  전차는 2654대와 679대  함정은 166척과 131척이라고 밝혔다. 양적으로는 한국이 많지만 질적 측면에선 일본이 우수하다는 것이 일반적인 평가이기도 하다.한국이 일본을 앞섰다는 GFP의 평가를 얼마나 

In [ ]:
print('#Train set size:', len(X_train))
print('#Test set size:', len(X_test))

#Train set size: 2312
#Test set size: 602


In [ ]:
from konlpy.tag import Okt 
okt = Okt()

print(okt.morphs(X_train[0])) # 모든 품사 반환 (형태소 토큰화)
print(okt.nouns(X_train[0])) # 명사만 반환
print(okt.pos(X_train[0])) # 품사 태깅

['한국', '전쟁', '수행', '능력', '일본', '을', '앞섰다', '.', '정욱식', '칼럼', '경제력', '차이', '3', '배', '국방', '예산', '과도', '하다는', '증거', '정욱식', '평화', '네트워크', '대표', '군사력', '측정', '전', '문', '기관', '인', '글로벌', '파이어', '파워', 'GFP', '가', '2018년', '잠재', '적', '인', '전쟁', '수행', '능력', '에서', '한국', '이', '일본', '을', '앞섰다는', '평가', '를', '내놨다', '.', '한국', '을', '136', '개', '평가', '대', '상국', '가운데', '7', '위로', '평가', '한', '반면', '일본', '은', '8', '위', '를', '기록', '했다는', '것', '이다', '.', '이', '기관', '이', '한국', '의', '전쟁', '수행', '능력', '을', '일본', '보다', '높게', '평가', '한', '것', '은', '2006년', '이래', '처음', '이다', '.', 'GFP', '는', '군사력', '의', '양', '과', '질적', '측면', '뿐', '만', '아니라', '자원', '재정', '지리', '산업', '등', '55', '개', '요소', '로', '해당', '국가', '의', '전쟁', '수행', '능력', '을', '평가', '해왔다', '.', '하지만', '핵무기', '보유', '고', '는', '중요한', '요소', '에', '포함', '시키지', '않았다', '.', '재래식', '군사력', '을', '동원', '한', '전쟁', '수행', '능력', '을', '주된', '평가', '대상', '으로', '삼았기', '때문', '이다', '.', '이', '에', '따라', '비공식', '핵보유국', '들', '인', '이스라엘', '파키스탄', '북한', '의', '전쟁', '수행', '능력', '은', '각

명사만 사용

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

#okt.nouns를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5) 

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000) 
clf.fit(X_train_tfidf, y_train) 
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test))) 

#Train set score: 0.987
#Test set score: 0.963


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

#okt.nouns를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=50, min_df=5, max_df=0.5) 

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000) 
clf.fit(X_train_tfidf, y_train) 
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) 
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test))) 

#Train set score: 0.923
#Test set score: 0.924


y_test : 실제값 X_test_tfidf : 예측값

for content in zip(y_test, clf.predict(X_test_tfidf)):
    print(content)

형태소 사용

In [ ]:
#okt.morphs를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) 
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.987
#Test set score: 0.973


In [ ]:
#okt.pos를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.pos, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) 
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.987
#Test set score: 0.967


최대 피쳐값 조정 (1000)

In [ ]:
#okt.morphs를 tokenizer로 이용 # 피쳐값 1000
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=1000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) 
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.985
#Test set score: 0.967


최대 피쳐값 조정 (200)

In [ ]:
#@title
#okt.morphs를 tokenizer로 이용 # 최대 피쳐값 200
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=200, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) 
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.972
#Test set score: 0.958


최소 빈도값 3

In [ ]:
#@title
#okt.morphs를 tokenizer로 이용 # 최소 빈도값 3
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=3, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) 
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.987
#Test set score: 0.973


최소빈도값 10

In [ ]:
#@title
#okt.morphs를 tokenizer로 이용 # 최소빈도값 10
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=10, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) 
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.987
#Test set score: 0.973


맥스iter 500

In [ ]:
#@title
#okt.morphs를 tokenizer로 이용 # 맥스iter 500
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=500)
clf.fit(X_train_tfidf, y_train)
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train))) 
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.987
#Test set score: 0.973


In [ ]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB(alpha=0.1)
NB_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

Train set score: 0.945
Test set score: 0.875


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators = 100, max_depth=30, random_state = 0)
model_rf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(model_rf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(model_rf.score(X_test_tfidf, y_test)))

Train set score: 1.000
Test set score: 0.988


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

model_ada = AdaBoostClassifier(n_estimators=100, base_estimator=DecisionTreeClassifier(max_depth=5),
                               algorithm='SAMME.R', learning_rate=0.5)
model_ada.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(model_ada.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(model_ada.score(X_test_tfidf, y_test)))

Train set score: 1.000
Test set score: 0.983


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators = 500, max_depth=15, random_state = 0)
model_rf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(model_rf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(model_rf.score(X_test_tfidf, y_test)))

Train set score: 1.000
Test set score: 0.990


In [ ]:
with open('logistic.pickle', 'wb') as f:
    pickle.dump(clf, f)
    
with open('logistic.pickle', 'rb') as f:
    model_pkl = pickle.load(f)